<div style="color:white;
           display:fill;
           border-radius:25px;
           background-color:Purple;
           font-size:210%;
           font-family:Verdana;
           letter-spacing:0.5px">
<p style="padding: 10px;
          color:white;
          text-align:center;"
          >
       WELCOME TO MY NOTEBOOK
</p>
</div>

# About Dataset: Tyre Quality Classification


![](https://i.gifer.com/fetch/w300-preview/bc/bcd3b446fcb8c0a210be67ab818e474b.gif)
![](https://i.gifer.com/origin/81/81311c16c69909b8327d81a9a9dfb176_w200.gif)



The dataset comprises 1854 digital images of tires, which are classified into two groups: one for **defective tires** and the other **for tires in good condition**. Each image is in a digital format and indicates the state of the tire, either as defective or in good condition.

In this notebook our main goal is to predict the Quality of the Tires based on input images. We use **Transfer Learning** approach to leverage pre-trained model **ResNet50** and enhance the performance of tire condition classification.

Thank you for exploring my notebook. Please take a moment to upvote my notebook. Your support motivates me to keep improving and sharing valuable insights.😊


# Import all the Libraries

In [ ]:
# import all the necsessary libraries
import pandas as pd
import os
import numpy as np
import seaborn as sns 
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,ConfusionMatrixDisplay,classification_report

# Lets do the Exploratory Data Analysis

In [ ]:
# Image Directory 
defective_images="/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective"
good_images="/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good"

In [ ]:
# See the number of files in Image Directory
defective_files= os.listdir(defective_images)
defective_files=len(defective_files)
print(f"No. of Defective Tyres images:{defective_files}")

In [ ]:
good_files= os.listdir(good_images)
good_files=len(good_files)
print(f"No. of Good Tyres images:{good_files}")

In [ ]:
# lets visualise the above numeric data
data = [defective_files, good_files]
keys = ["Defective_Tyres", "Good_Tyres"]

plt.bar(keys,data)
plt.xlabel("Tyre Condition")
plt.ylabel("Count")
plt.title("Tyre Condition Count Plot")
plt.show()


# Lets Divide the dataset into Trainig and Validation Set

In [ ]:
image_directory="/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres"
batch_size=32
img_width=224
img_height=224

train_data = tf.keras.utils.image_dataset_from_directory(image_directory,
                                                         validation_split=0.2,
                                                         seed=123,
                                                         subset="training",
                                                         image_size=(img_height, img_width),
                                                         batch_size=batch_size)

In [ ]:
# Print the class names
class_names=train_data.class_names
print(class_names)

In [ ]:
# Lets create the Validation data 
val_data = tf.keras.utils.image_dataset_from_directory(image_directory,
                                                       validation_split=0.2,
                                                       seed=123,
                                                       subset="validation",
                                                       image_size=(img_height, img_width),
                                                       batch_size=batch_size)

# Lets Visualize the Images

In [ ]:
for image, label in train_data.take(1):
    print(f"Shape of Images with batch_size:{image.shape}")
    print("----------------------------------------------------------")
    print(f"Total no. of labels contain in one batch:{label.shape}")
    print("----------------------------------------------------------")
    print(f"Check the single image in form of tensors:{image[0]}")
    print("------------------------------------------------------------")
    print(f"Check the label of same image:{class_names[label[0]]}")

In [ ]:
# Plot the images
plt.figure(figsize=(20,20), tight_layout=True, facecolor="white")
for image, label in train_data.take(1):
    for i in range(32):
        ax=plt.subplot(8, 4, i+1)
        plt.imshow(image[i].numpy().astype("uint8"))
        plt.title(class_names[label[i]])
        plt.axis("off")

# For Model Training we use Transfer Learning

In [ ]:
# Create base model with tf.keras.applications
base_model =tf.keras.applications.resnet.ResNet50(include_top=False)


# Freeze the base model (so the pre-learned patterns remain same)
base_model.trainable = False


# Create inputs into the base model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")
x = base_model(inputs)


# Check data shape after passing it to base_model
print(f"Shape after base_model: {x.shape}")


# Average pool the outputs of the base model (aggregate all the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)



x=tf.keras.layers.Flatten()(x)
x=tf.keras.layers.Dense(512, activation="relu")(x)
#x=tf.keras.layers.Dropout(0.5)(x)

# Create the output activation layer
outputs=tf.keras.layers.Dense(1, activation="sigmoid",name="output_layer")(x)

# Combine the inputs with the outputs into a model
model= tf.keras.Model(inputs, outputs, name="model")



# Compile the model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),   # different loss function for Binary classifcation
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=["accuracy"]
             )


# Callbacks--> Stop training automatically once the model performance stop improving
learning_rate_reduction=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",patience=2,factor=0.5, min_lr=0.00001,verbose =1)
Early_Stopping= tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=5, restore_best_weights=True)

# Fit the model
history1= model.fit(train_data, epochs=5, validation_data=val_data, callbacks=[Early_Stopping,learning_rate_reduction], verbose=1)

# Model Summary

In [ ]:
model.summary()

# Lets Plot the Model

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, show_dtype=True, dpi=120)

# Lets Plot the History Curve

In [ ]:
dataframe=pd.DataFrame(history1.history)
dataframe

# Lets plot the Training and Validation Loss Curves

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.plot(dataframe["loss"], label="Training_Loss")
plt.plot(dataframe["val_loss"], label="Validation_Loss")
plt.title('Train_Loss and Validation_Loss',fontsize=15)
plt.legend()

plt.subplot(1,2,2)
plt.plot(dataframe["accuracy"], label="Training_Accuracy")
plt.plot(dataframe["val_accuracy"], label="Validation_Accuracy")
plt.title('Train_Accuracy and Validation_Accuracy',fontsize=15)
plt.legend()
plt.show()

# Lets Evaluate the Model

In [ ]:
loss, accuracy=model.evaluate(val_data)
print(f"Accuracy:{accuracy}")
print(f"Loss:{loss}")

# Lets make the Predictions

In [ ]:
x_val=[]
y_val=[]


for image, label in val_data:
    x_val.extend(image.numpy())
    y_val.extend(label.numpy())    

In [ ]:
# Lets make the Predictions
predictions=model.predict(np.array(x_val))

y_pred=[int(np.round(prediction)) for prediction in predictions]

In [ ]:
plt.figure(figsize=(35, 35),tight_layout=True, facecolor="white")


for i in range(32):
    ax = plt.subplot(8, 4, i + 1)
    plt.imshow(x_val[i].astype("uint8"))
    actual_label = class_names[y_val[i]]
    predicted_label = class_names[y_pred[i]]
    
    # Check if the actual and predicted labels are the same
    if actual_label == predicted_label:
        label_color ='green'
    else:
        label_color = 'red'
    
     
    plt.title(f'Actual: {actual_label} \n Predicted: {predicted_label}', color=label_color,fontsize=20)
    plt.axis("off")

plt.show()

# Results

In [ ]:
f1_scores=[]
precision_scores=[]
recall_scores=[]

f1_scores.append(f1_score(y_pred, y_val, average="macro"))
precision_scores.append(precision_score(y_pred, y_val, average="macro"))
recall_scores.append(recall_score(y_pred, y_val, average="macro"))


# Print the Results
print(f"F1-Score:{f1_scores}")
print(f"Precision:{precision_scores}")
print(f"Recall:{recall_scores}")



print("Classification_Report")
print("-----------------------")
print(classification_report(y_val,y_pred))
print("Confusion_Matrix")
print("----------------------")
ConfusionMatrixDisplay.from_predictions(y_val, y_pred, display_labels=class_names, xticks_rotation="vertical")
plt.show()

# Lets Visualise the Final Results

In [ ]:
model_name=["ResNet50"]
result_df= pd.DataFrame({"F1_Score":f1_scores, "Precision_Score":precision_scores, "Recall_Score":recall_scores}, index=model_name)
result_df= result_df.T.sort_values(by="ResNet50", ascending=False)
result_df

In [ ]:
result_df.plot(kind="bar",figsize=(5,5), color="Green").legend(bbox_to_anchor=(1.5,1))